# Redes Neurais

Esse notebook tratará da implementação, do treino e da avaliação inicial de redes neurais recorrentes (especificamente de células LSTM) na busca pela solução do seguinte problema de classificação: identificação de de viés político em textos. O tamanho desses textos é algo a ser arbitrado e até mesmo utilizado como comparação: podemos utilizar somente as sentenças que foram marcadas por conter viés, ou mesmo o texto na íntegra da notícia.

In [57]:
import numpy as np
import pickle
import ast
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit 
from tensorflow import keras
import tensorflow as tf

- Carregamento do dataset e do vocabulário, totalmente perfilados pelo notebook de pré-processamento.

In [58]:
data = pd.read_csv('../data/final_dataset.csv')

# transforma as listas que estão em string em listas novamente
data["sentence"] = data["sentence"].apply(eval)
data["article"] = data["article"].apply(eval)
data["biased_words4"] = data["biased_words4"].apply(eval)

In [59]:
# Transforma as listas em Arrays do Numpy
data["sentence"] = data["sentence"].apply(np.asarray)
data["article"] = data["article"].apply(np.asarray)
data["biased_words4"] = data["biased_words4"].apply(np.asarray)

In [60]:
data.columns

Index(['sentence', 'outlet_alternet', 'outlet_breitbart', 'outlet_federalist',
       'outlet_fox-news', 'outlet_huffpost', 'outlet_msnbc', 'outlet_reuters',
       'outlet_usa-today', 'topic_abortion', 'topic_coronavirus',
       'topic_elections-2020', 'topic_environment', 'topic_gender',
       'topic_gun-control', 'topic_immigration',
       'topic_international-politics-and-world-news', 'topic_middle-class',
       'topic_sport', 'topic_student-debt', 'topic_trump-presidency',
       'topic_vaccines', 'topic_white-nationalism', 'type_center', 'type_left',
       'type_right', 'num_sent', 'Label_bias_Biased',
       'Label_bias_No agreement', 'Label_bias_Non-biased', 'article',
       'biased_words4'],
      dtype='object')

In [61]:
with open('../data/vocabulary_table.pkl', 'rb') as f:
    vocab_dict = pickle.load(f)

keys = list(vocab_dict.keys())
values = list(vocab_dict.values())
n_oov = 5000

vocab_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(keys, values, tf.string, tf.int64),
    n_oov
)


## 1 - Separando o dataset

In [62]:
label_columns = ['Label_bias_Biased','Label_bias_No agreement', 'Label_bias_Non-biased']
labels = data[label_columns]
features = data.drop(label_columns, axis=1)

In [63]:
def train_valid_test_split(features, labels):
    """ Retorna uma lista de tuplas contendo os datasets de features e de labels para cada segmento (treino, validação, teste) """

    # Treino-val e Teste
    shuffle_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=892)
    train_val_indexes, test_indexes = next(shuffle_train_test.split(features.values, labels.values))
    train_val_df, train_val_labels = features.iloc[train_val_indexes], labels.iloc[train_val_indexes]
    test_df, test_labels = features.iloc[test_indexes], labels.iloc[test_indexes]

    # Treino e Validação
    shuffle_train_validate = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=124)
    train_indexes, validation_indexes = next(shuffle_train_validate.split(train_val_df.values,train_val_labels.values))
    train_df, train_labels = features.iloc[train_indexes], labels.iloc[train_indexes]
    validation_df, validation_labels = features.iloc[validation_indexes], labels.iloc[validation_indexes]


    return [(train_df, train_labels), (validation_df, validation_labels), (test_df, test_labels)]

In [64]:
train, validation, test = train_valid_test_split(features,labels)

train_df, train_labels = train
validation_df, validation_labels = validation
test_df, test_labels = test

print(f"Treino:{train_df.shape} ----- Validação: {validation_df.shape} ------ Teste: {test_df.shape}")

Treino:(1014, 29) ----- Validação: (338, 29) ------ Teste: (338, 29)


## 2 - Estruturando arquitetura da rede neural

In [65]:
embed_size = 128

# parte textual
text_input_length = len(train_df.loc[0,'article']) + len(train_df.loc[0,'biased_words4']) + len(train_df.loc[0,'sentence'])
text_input = keras.layers.Input(shape=(text_input_length,))
embedding_layer = keras.layers.Embedding(input_dim=int(vocab_table.size())+5000,
                                         output_dim=embed_size,
                                         mask_zero=True)(text_input)

embedding_output = keras.layers.Bidirectional(keras.layers.LSTM(embed_size))(embedding_layer) 


# outras features 
feature_input_length = len(train_df.columns) - 3
feature_input = keras.layers.Input(shape=(feature_input_length,))

concatenated = keras.layers.concatenate([embedding_output,feature_input], axis=-1)
reshaped = keras.layers.Reshape((-1, concatenated.shape[-1]))(concatenated)  # Add time dimension
gru_layer1 = keras.layers.GRU(64)(reshaped)
output = keras.layers.Dense(3, activation="softmax")(gru_layer1)

model = tf.keras.Model(inputs=[text_input,feature_input], outputs=output)
#model.summary()
model.compile(loss="categorical_crossentropy",optimizer="adam")


In [69]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 3066)]               0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 3066, 128)            4588032   ['input_9[0][0]']             
                                                                                                  
 bidirectional_4 (Bidirecti  (None, 256)                  263168    ['embedding_4[0][0]']         
 onal)                                                                                            
                                                                                                  
 input_10 (InputLayer)       [(None, 26)]                 0         []                      

In [55]:
text_cols = ['sentence','article','biased_words4']
others = list(filter(lambda x: x if x not in text_cols else [],train_df.columns))

train_df = pd.concat([train_df[text_cols], train_df[others]],axis=1)
validation_df = pd.concat([validation_df[text_cols], validation_df[others]],axis=1)
test_df = pd.concat([test_df[text_cols], test_df[others]],axis=1)

In [37]:
train_df.sentence

93      [1, 13361, 843, 5621, 172, 6203, 408, 566, 242...
526     [1, 1434, 1039, 2681, 1397, 701, 281, 304, 136...
761     [1, 1472, 80, 3802, 191, 701, 254, 4393, 11330...
686     [1, 4711, 12583, 3519, 489, 14024, 2248, 485, ...
1067    [1, 12645, 145, 154, 145, 2301, 1043, 63, 1643...
                              ...                        
1047    [1, 888, 1787, 3796, 966, 13877, 13878, 17365,...
1327    [1, 9015, 1567, 3828, 1678, 17015, 644, 2395, ...
1302    [1, 522, 1081, 458, 4056, 6374, 304, 3236, 162...
245     [1, 864, 776, 421, 168, 265, 4140, 3470, 1775,...
1000    [1, 248, 1114, 3765, 3761, 17010, 145, 513, 88...
Name: sentence, Length: 1014, dtype: object

In [38]:
model.fit(train_df, train_labels, batch_size=32, epochs=10, validation_data=(validation_df, validation_labels))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).